In [85]:
import numpy as np
import matplotlib.pyplot as plt
from ansys.mapdl.core import launch_mapdl
mapdl=launch_mapdl()
print(mapdl)

Product:             Ansys Mechanical Enterprise Academic Student
MAPDL Version:       23.1
ansys.mapdl Version: 0.64.0



In [86]:
mapdl.clear()
mapdl.prep7()


*** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2023 R1          23.1     ***
 Ansys Mechanical Enterprise Academic Student      
 01055371  VERSION=WINDOWS x64   02:14:28  APR 05, 2023 CP=      0.109

                                                                               



          ***** MAPDL ANALYSIS DEFINITION (PREP7) *****

In [87]:
# Define Material Property
E = 210e9
nu = 0.3
YS = 240e6
# Defining Geometry
a = 0.01 # Crack length
c = 0.03 # Thickness

In [88]:

m0 = mapdl.k("", 0, -0.1)
m1 = mapdl.k("", 0,0.1005)
m2 = mapdl.k("",0.005,0.1005)
m3 = mapdl.k("",0.005,0.1)
m4 = mapdl.k("", 0.1, 0.1)
m5 = mapdl.k("", 0.1, -0.1)
main = mapdl.a(m0,m1,m2,m3,m4,m5)
k0 = mapdl.k("",0, -0.0001)
k1 = mapdl.k("", 0.01,0)
k2 = mapdl.k("", 0,0.0001)
crack = mapdl.a(k0, k1, k2)
aout = mapdl.asba(main, crack)
mapdl.vext(aout, dz=0.005)
mapdl.vplot(show_lines=True, quality=5)

Widget(value="<iframe src='http://localhost:53378/index.html?ui=P_0x1bcae5cceb0_16&reconnect=auto' style='widt…

In [94]:
mapdl.et(1,"SOLID285")
#mapdl.et(1, 285)
mapdl.smrtsize(1)
mapdl.aesize("ALL", 0.001)
mapdl.vmesh(1)
scon1 = mapdl.k(10000,0.1,0,0)
scon2 = mapdl.k(10001,0.2,0,0)
mapdl.krefine(1,2,1)
mapdl.eplot(vtk=True, show_edges=True, show_axes=False, line_width=2, background="w")

Widget(value="<iframe src='http://localhost:53378/index.html?ui=P_0x1bcc690cd60_18&reconnect=auto' style='widt…

PPATH  Point=  2  Node=     191  X,Y,Z= 0.9400E-02 0.6003E-05 0.5000E-02 cs= 11

In [19]:
mapdl.open_gui()

In [54]:
# This example will use SI units.
mapdl.units("SI")  # SI - International system (m, kg, s, K).

# Define a material (nominal steel in SI)
mapdl.mp("EX", 1, 210e9)  # Elastic moduli in Pa (kg/(m*s**2))
mapdl.mp("DENS", 1, 7800)  # Density in kg/m3
mapdl.mp("NUXY", 1, 0.3)  # Poisson's Ratio

# Fix the lower side.
mapdl.nsel("S", "LOC", "Y", -0.1)
mapdl.d("ALL", "UY")
mapdl.d("ALL", "UZ")
mapdl.d("ALL", "UX")

mapdl.nsel("S", "LOC", "Y", 0.1)
mapdl.cp(5, "UX", "ALL")

#p0 = mapdl.k("", 0, 0.1, 0)
#p1 = mapdl.k("", 0, 0.1, 0.005)
#p2 = mapdl.k("", 0.005, 0.1, 0.005)
#p3 = mapdl.k("",0.005,0.1,0)
#loadArea = mapdl.a(p0,p1,p2,p3)
#mapdl.asel("S","AREA",vmin = loadArea,vmax = loadArea)
mapdl.asel("S",'LOC',"Y",0.1005)
mapdl.sfa('ALL', '', 'PRES', -250e6)
#mapdl.f("ALL", "FY", 10000)
mapdl.allsel(mute=True)

In [55]:
mapdl.run("/SOLU")
mapdl.antype("STATIC")
mapdl.solve()
mapdl.finish(mute=True)

In [56]:
# grab the result from the ``mapdl`` instance
result = mapdl.result
result.plot_principal_nodal_stress(
    0,
    "SEQV",
    lighting=False,
    background="w",
    show_edges=True,
    text_color="k",
    add_text=False,
)

nnum, stress = result.principal_nodal_stress(0)
von_mises = stress[:, -1]  # von-Mises stress is the right most column

# Must use nanmax as stress is not computed at mid-side nodes
max_stress = np.nanmax(von_mises)

Widget(value="<iframe src='http://localhost:53378/index.html?ui=P_0x1bc8d0b6aa0_7&reconnect=auto' style='width…

In [60]:
mapdl.post1()
mapdl.path('P',3,5)
node1 = mapdl.queries.node(0.01,0,0)
node2 = mapdl.queries.node(0.009,0.0002,0)
node3 = mapdl.queries.node(0.006,0.0002,0)
node2

48

In [61]:
mapdl.ppath(1, node1)
mapdl.ppath(2, node2)
mapdl.ppath(3, node3)

a = mapdl.kcalc(0,1,0,1)

In [62]:
words = a.split(' ')
# Extracting the K1 value from the String output
K1 = words[307][:-1]
print(f"K1 is equal to {K1}")

K1 is equal to 


In [16]:
sif = mapdl.post_processing.nodal_stress_intensity()
maxSIF = np.max(sif)

In [17]:
maxSIF

929285836.2052578

In [57]:
# We use nanmean here because mid-side nodes have no stress
mask = result.mesh.nodes[:, 0] == 0.1
far_field_stress = np.nanmean(von_mises[mask])
print("Far field von mises stress: %e" % far_field_stress)
# Which almost exactly equals the analytical value of 10000000.0 Pa

Far field von mises stress: 2.266953e+08


In [58]:
stress_con = max_stress / far_field_stress
print("Stress Concentration: %.2f" % stress_con)

Stress Concentration: 3.65


In [59]:
sif = mapdl.post_processing.nodal_stress_intensity()
maxSIF = np.max(sif)

In [60]:
sif

array([3.45556478e+08, 3.79327708e+06, 2.67505553e+08, ...,
       2.54322097e+08, 2.61947275e+08, 2.33798139e+08])

In [61]:
maxSIF =np.max(sif)
maxSIF

929285836.2052511